In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
df = pd.read_csv("../input/train.csv") 
df_test = pd.read_csv("../input/test.csv")

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Print all rows and columns. Dont hide any
##df.drop = ('id', axis = 1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.head(10)

###Data Cleaning/Feature Engineering
Using a method from Alexandru's kernel, we identified the columns with the greatest number of unique categorical variables and applied factorization to them (in order to reduce memory demand and runtime). We then converted the columns with 20 or fewer categorical variables into dummies/indicator variables. However, running the model with this transformed data actually increases the error, so we do not incorporate this in the final models. 

In [ ]:
##identify the columns with the most parameters and apply factorization to them (code from Alexandru)
df.iloc[:,1:116].apply(lambda x: pd.unique(x).shape[0]).sort_values(ascending = False).head(10)

In [ ]:
train_cat_large = df[["cat116", "cat110","cat109","cat113", "cat112", "cat115"]]
df_frac = df.drop(train_cat_large, axis = 1)
X_int_cat = train_cat_large.apply(lambda x: pd.factorize(x)[0])
df = pd.concat([df_frac, X_int_cat], axis = 1)
#new dataframe factorizes the columns with most variables

In [ ]:
df.head()

num_train = df.size
all_data = pd.concat((df, df_test)).reset_index(drop=True)
features = df.columns
cats = [feat for feat in features if "cat" in feat]
for feat in cats:
    all_data[feat] = pd.factorize(all_data[feat])[0]
df_train = all_data.iloc[:num_train]
df_test = all_data.iloc[num_train:]

In [ ]:
df = pd.get_dummies(df, dummy_na = False, drop_first = True)
df.tail()
print(df.shape)

In [ ]:
col = list(df.columns)
#numcol = col[:15]
df['log_loss'] = np.log1p(df['loss'])
df.head()

In [ ]:
import seaborn as sns
sns.lmplot('cont3' , 'log_loss', data = df)

Scatterplot for one of the continuous variables and the log-transformed loss. The best-fit line indicates that there may be a slight positive correlation. 

##Violin Plots for Categorical Variables
These plots show the distribution of the log-transformed losses with respect to each variable in the categorical columns. The distributions for different variables within each columns generally seem pretty similar, indicating that most attributes by themselves are not a good indicator of loss.

In [ ]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
import statsmodels.api as sm

In [ ]:
##Split data and log transform
local_train, local_test = train_test_split(df,test_size=0.2,random_state=123)
df_y_train = np.logp1(local_train['loss'])
df_x_train = local_train.drop(["loss"],axis=1)
df_y_test = np.logp1(local_test['loss'])
df_x_test = local_test.drop(["loss"],axis=1)

In [ ]:
##Training on linear Regression
clf = sm.OLS(df_y_train, df_x_train)
result = clf.fit()
preds = result.predict(df_x_test)
preds

#Linear Regression Model
As a first run, this model returns an error value above the benchmark. It may be less accurate due to the high correlation between many of the continuous variables. 